In [1]:
'''
    modified to simplify some operations
    
    Note: accepts a .gz file and a col idx for the text to process
    
    @author rosecatherinek
    @modified Jan 23, 2017
'''

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import pickle, sys, gzip
import random
import numpy as np
import tensorflow as tf


#Settings:
# input_file = sys.argv[1]  #gz file
# col = int(sys.argv[2])
outfile_embedding = "word_emb.pkl"
outfile_dict = "word_dict.pkl"
outfile_dict_rev = "rev_dict.pkl"

embedding_size = 50    # Dimension of the embedding vector.
num_steps = 1000001


In [2]:
import sqlite3

conn = sqlite3.connect('yelp_dataset.db')
c = conn.cursor()

def Select(q):
    return c.execute(q)

def Update(q):
    c.execute(q)
    conn.commit()
    
        
# conn.close()
query = Select("Select review_text from Reviews")


In [4]:
import collections
import string 
punc = set(string.punctuation)
words = []
i = 0
gc = collections.Counter([])
for row in query:
    if i % 10000 == 0:
        print (i,end=" "),
        c = collections.Counter(words)
        gc += c
        words = []
    text = row[0]
    rev_w = text.lower().split()
    for w in rev_w:
        prev = 0
        for ch in range(len(w)):
            if w[ch] in punc:
                if prev < ch:
                    words += [w[prev:ch]]
                words += [w[ch]]
                prev = ch + 1
        if prev < len(w):
            words += [w[prev:]]
    i += 1
gc += collections.Counter(words)
    
    

0 10000 20000 30000 40000 50000 60000 70000 80000 90000 100000 110000 120000 130000 140000 150000 160000 170000 180000 190000 200000 210000 220000 230000 240000 250000 260000 270000 280000 290000 300000 310000 320000 330000 340000 350000 360000 370000 380000 390000 400000 410000 420000 430000 440000 450000 460000 470000 480000 490000 500000 510000 520000 530000 540000 550000 560000 570000 580000 590000 600000 610000 620000 630000 640000 650000 660000 670000 680000 690000 700000 710000 720000 730000 740000 750000 760000 770000 780000 790000 800000 810000 820000 830000 840000 850000 860000 870000 880000 890000 900000 910000 920000 930000 940000 950000 960000 970000 980000 990000 1000000 1010000 1020000 1030000 1040000 1050000 1060000 1070000 1080000 1090000 1100000 1110000 1120000 1130000 1140000 1150000 1160000 1170000 1180000 1190000 1200000 1210000 1220000 1230000 1240000 1250000 1260000 1270000 1280000 1290000 1300000 1310000 1320000 1330000 1340000 1350000 1360000 1370000 1380000 13

In [7]:
# import string 
# punc = set(string.punctuation)
# w = "stop.it!!"
# words = []
# prev = 0
# for ch in range(len(w)):
#     if w[ch] in punc:
#         if prev < ch:
#             words += [w[prev:ch]]
#         words += [w[ch]]
#         prev = ch +1
# if prev < len(w):
#     words += [w[prev:]]
# # words += [w[prev:]]
# print(words)
gc.most_common(2)
del words

In [10]:
len(gc.keys())

TypeError: 'dict_keys' object does not support indexing

In [12]:
# print('Data size', len(words))

# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 200000


def build_dataset(c):
    count = [['<UNK>', -1]]
    count.extend(c.most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, cnt in count:
        if (not word == '<UNK>') and cnt < 3:
            continue
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    words = list(gc.keys())
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

print ('Building dictionary...')

data, count, dictionary, reverse_dictionary = build_dataset(gc)

#update the size of vocab
vocabulary_size = min(vocabulary_size, len(dictionary))

print ('vocabulary size = ', vocabulary_size)

# del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])


Building dictionary...
vocabulary size =  200000
Most common words (+UNK) [['<UNK>', 495429], ('.', 39193930), ('the', 27003268), (',', 18948024), ('and', 18530059)]
Sample data [156082, 197625, 0, 197626, 17145, 0, 143288, 0, 21546, 0] ['bullfighter', 'mallin', '<UNK>', 'edamamae', 'puking', '<UNK>', 'borage', '<UNK>', 'upsale', '<UNK>']


In [14]:

data_index = 0


# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)

for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0], reverse_dictionary[labels[i, 0]])


# Step 4: Build and train a skip-gram model.
batch_size = 128
skip_window = 1         # How many words to consider left and right.
num_skips = 2           # How many times to reuse an input to generate a label.


# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():
    
    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    # Look up embeddings for inputs.
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup( normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

# Step 5: Begin training.

cfg = tf.ConfigProto(allow_soft_placement=True )
cfg.gpu_options.allow_growth = True
with tf.Session(graph=graph, config=cfg) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
    
        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0
    
        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
#         if step % 1000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = "Nearest to %s:" % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    
    final_embeddings = normalized_embeddings.eval()
    
    

    # Step 6: Save the dictionary & the embeddings
    print ("Pickling embeddings and dict")
    pickle.dump( final_embeddings, open( outfile_embedding , "wb" ) )
    pickle.dump( dictionary, open(outfile_dict, "wb"))
    pickle.dump( reverse_dictionary, open(outfile_dict_rev, "wb"))
    
    print ('Done.')


197625 mallin -> 156082 bullfighter
197625 mallin -> 0 <UNK>
0 <UNK> -> 197625 mallin
0 <UNK> -> 197626 edamamae
197626 edamamae -> 17145 puking
197626 edamamae -> 0 <UNK>
17145 puking -> 0 <UNK>
17145 puking -> 197626 edamamae
Initialized
Average loss at step  0 :  347.186584473
Nearest to do: traminette, oars, herausgekommen, ceaser, urad, inion, urned, kiran,
Nearest to back: varried, submarine, contaminated, meriter, jabawokeez, entpuppte, hamid, sloooooow,
Nearest to have: m5t, highend, neighbirhood, ime, buuuutttt, flyknit, whammed, aleksander,
Nearest to your: guarranteed, briefings, hokki, ratleff, messes, unaufmerksames, scally, hakasaan,
Nearest to at: budgies, okeke, clinical, gyozo, astrological, schlage, ccrepairz, eral,
Nearest to ': doerr, rient, harboured, carrol, aw, pastime, foundue, scaffold,
Nearest to up: bogof, lobbing, louisville, menor, heidi, sitte, 13k, dweeb,
Nearest to t: grannie, bfc, fizzed, capitán, pinty, indestructible, kenedi, paniermehl,
Nearest to wi

Average loss at step  42000 :  42.1206404616
Average loss at step  44000 :  45.8988537495
Average loss at step  46000 :  52.9466210113
Average loss at step  48000 :  47.4006259677
Average loss at step  50000 :  48.0334129379
Nearest to do: traminette, kiran, ceaser, christmastime, urad, oars, inion, swingset,
Nearest to back: submarine, dies, varried, contaminated, jabawokeez, meriter, cooks, entpuppte,
Nearest to have: buuuutttt, neighbirhood, m5t, flyknit, alsonso, ime, goeke, amin,
Nearest to your: es, guarranteed, hokki, maras, sidled, scally, rhevp5flf6koprimhmvygg, zind,
Nearest to at: okeke, bleomycin, budgies, sherelle, wilmington, clinical, gyozo, sequestered,
Nearest to ': doerr, rient, aw, carrol, harboured, finnz, scaffold, euros,
Nearest to up: bogof, dweeb, lobbing, louisville, atlanta, habían, duffrin, heidi,
Nearest to t: grannie, fizzed, tradición, capitán, indestructible, pinty, bfc, frenemies,
Nearest to will: dahn, centered, winzer, mcgovern, évolue, lnsp, artigiani

Average loss at step  94000 :  62.3290450034
Average loss at step  96000 :  72.453940942
Average loss at step  98000 :  83.3984775381
Average loss at step  100000 :  70.7743404324
Nearest to do: traminette, kiran, ceaser, urned, urad, oars, christmastime, swingset,
Nearest to back: dies, submarine, varried, meriter, contaminated, jabawokeez, cooks, overdrafted,
Nearest to have: neighbirhood, buuuutttt, m5t, ime, flyknit, fcp, nehme, balley,
Nearest to your: es, guarranteed, sidled, hokki, maras, acorn, rhevp5flf6koprimhmvygg, scally,
Nearest to at: okeke, bleomycin, republica, budgies, eral, gyozo, sherelle, cumcumbers,
Nearest to ': doerr, carrol, rient, aw, harboured, pastime, foundue, euros,
Nearest to up: bogof, lobbing, shaping, dweeb, louisville, menor, heidi, atlanta,
Nearest to t: grannie, fizzed, capitán, tradición, bfc, indestructible, sister, frenemies,
Nearest to will: dahn, winzer, centered, mcgovern, évolue, lnsp, florentia, artigiani,
Nearest to $: fiction, quietly, conq

Average loss at step  150000 :  75.3486664807
Nearest to do: traminette, kiran, oars, ceaser, mows, urned, swingset, urad,
Nearest to back: submarine, dies, meriter, varried, contaminated, cooks, jabawokeez, crumbly,
Nearest to have: buuuutttt, neighbirhood, m5t, balley, ime, parlours, fcp, amin,
Nearest to your: guarranteed, es, sidled, hokki, maras, scally, acorn, heathly,
Nearest to at: okeke, bleomycin, budgies, republica, sherelle, eral, collab, astrological,
Nearest to ': doerr, carrol, harboured, aw, rient, pastime, foundue, euros,
Nearest to up: bogof, lobbing, shaping, menor, atlanta, dweeb, habían, belljar,
Nearest to t: grannie, fizzed, tradición, paniermehl, sister, bfc, flophouse, indestructible,
Nearest to will: dahn, winzer, centered, lnsp, évolue, bollo, artigiani, signifie,
Nearest to $: fiction, quietly, conquérir, tipsey, giel, bristly, preened, vaporizing,
Nearest to food: caled, vapid, studebaker, unique, byrider, ultrastar, remnant, argumentation,
Nearest to .: wa

Average loss at step  200000 :  69.9337245828
Nearest to do: traminette, kiran, oars, lauriston, urned, swingset, ceaser, christmastime,
Nearest to back: submarine, dies, varried, contaminated, meriter, jabawokeez, shhhhhhhh, hamid,
Nearest to have: buuuutttt, m5t, neighbirhood, balley, parlours, ime, alsonso, fcp,
Nearest to your: es, guarranteed, sidled, hokki, rhevp5flf6koprimhmvygg, maras, heathly, acorn,
Nearest to at: budgies, bleomycin, okeke, eral, sherelle, republica, collab, gyozo,
Nearest to ': doerr, carrol, aw, harboured, pastime, foundue, rient, euros,
Nearest to up: bogof, shaping, lobbing, atlanta, heidi, menor, jook, trato,
Nearest to t: grannie, fizzed, tradición, sister, capitán, indestructible, flophouse, paniermehl,
Nearest to will: winzer, dahn, centered, mcgovern, lnsp, évolue, signifie, monopolstellung,
Nearest to $: fiction, quietly, conquérir, bristly, tipsey, giel, vaporizing, preened,
Nearest to food: caled, argumentation, vapid, ultrastar, unique, byrider, 

...

Average loss at step  802000 :  26.1856652308
Average loss at step  804000 :  24.7303492991
Average loss at step  806000 :  26.695489985
Average loss at step  808000 :  23.9475906137
Average loss at step  810000 :  24.1237563223
Nearest to do: traminette, lauriston, oars, kiran, urned, mows, appetitite, yazmin,
Nearest to back: dies, x36, meriter, greeted, crumbly, sey, ethipian, maintaned,
Nearest to have: m5t, buuuutttt, balley, ime, fortifying, parlours, neighbirhood, tossin,
Nearest to your: guarranteed, sidled, es, bonatz, scally, heathly, nuanced, acorn,
Nearest to at: budgies, sherelle, eral, hinzusetzen, republica, dardanelles, cumcumbers, vineyard,
Nearest to ': doerr, carrol, harboured, pastime, rient, mascaras, aw, dameron,
Nearest to up: shaping, jook, bogof, loaning, eiw, hallucinogenics, menor, verwandelt,
Nearest to t: grannie, fizzed, sister, indestructible, beaten, idles, galletas, paniermehl,
Nearest to will: signifie, mcgovern, dahn, bollo, winzer, lnsp, foodigatorj,

Average loss at step  852000 :  22.3676957953
Average loss at step  854000 :  22.4837020269
Average loss at step  856000 :  25.4697484344
Average loss at step  858000 :  22.1951178481
Average loss at step  860000 :  21.4896900951
Nearest to do: traminette, lauriston, kiran, oars, urned, mows, appetitite, yazmin,
Nearest to back: dies, x36, meriter, greeted, crumbly, sey, maintaned, ethipian,
Nearest to have: m5t, buuuutttt, balley, ime, parlours, fortifying, ansprüche, goubuli,
Nearest to your: guarranteed, sidled, es, bonatz, heathly, scally, nuanced, ratleff,
Nearest to at: budgies, sherelle, eral, hinzusetzen, republica, dardanelles, vineyard, collab,
Nearest to ': doerr, carrol, harboured, pastime, rient, mascaras, aw, scaffold,
Nearest to up: jook, shaping, eiw, loaning, bogof, hallucinogenics, menor, verwandelt,
Nearest to t: grannie, fizzed, sister, indestructible, beaten, idles, capitán, flophouse,
Nearest to will: signifie, dahn, mcgovern, bollo, lnsp, monopolstellung, winzer,

Average loss at step  902000 :  23.4810885197
Average loss at step  904000 :  20.9813094933
Average loss at step  906000 :  24.442868191
Average loss at step  908000 :  23.6515797639
Average loss at step  910000 :  20.8793290207
Nearest to do: traminette, lauriston, kiran, oars, urned, mows, appetitite, yazmin,
Nearest to back: dies, x36, meriter, greeted, sey, maintaned, crumbly, giovani,
Nearest to have: m5t, buuuutttt, balley, ime, parlours, fortifying, restaurnat, unaccompanied,
Nearest to your: guarranteed, sidled, bonatz, es, ratleff, heathly, nuanced, scally,
Nearest to at: budgies, sherelle, eral, hinzusetzen, dardanelles, republica, vineyard, collab,
Nearest to ': doerr, carrol, pastime, harboured, mascaras, aw, scaffold, rient,
Nearest to up: jook, shaping, eiw, loaning, bogof, hallucinogenics, verwandelt, menor,
Nearest to t: grannie, fizzed, sister, beaten, indestructible, exaggerates, idles, galletas,
Nearest to will: signifie, dahn, mcgovern, lnsp, bollo, monopolstellung,

Nearest to more: daytripper, zubehör, rishi, ett, scarlet, servers, parhizgar, imports,
Nearest to don: nals, rustle, entertaining, pesticide, improviser, onlookers, andrey, danged,
Nearest to on: fishstick, jaspers, plust, operational, duddingston, contain, crzy, qqj8xmca49wsu1jxiwwwdq,
Average loss at step  952000 :  22.3983687011
Average loss at step  954000 :  21.5269180338
Average loss at step  956000 :  20.299451286
Average loss at step  958000 :  23.1972318373
Average loss at step  960000 :  24.8284336572
Nearest to do: traminette, lauriston, kiran, oars, urned, appetitite, yazmin, atest,
Nearest to back: dies, x36, meriter, greeted, maintaned, sey, crumbly, giovani,
Nearest to have: m5t, buuuutttt, balley, parlours, ime, fortifying, unaccompanied, blatantly,
Nearest to your: guarranteed, sidled, bonatz, es, ratleff, heathly, acorn, nuanced,
Nearest to at: budgies, sherelle, eral, dardanelles, hinzusetzen, republica, vineyard, épurée,
Nearest to ': doerr, carrol, pastime, harbou

Nearest to on: fishstick, jaspers, plust, operational, duddingston, qqj8xmca49wsu1jxiwwwdq, crzy, legalization,
Pickling embeddings and dict
Done.
